In [1]:
import json
from data_prep import match_components, convert_matched_components_to_compressed_string

components_file_path = 'Prompt_Experiment/components.json'
screens_file_path = 'Prompt_Experiment/400screens.json'

with open(components_file_path, 'r') as file:
    components_data = json.load(file)
with open(screens_file_path, 'r') as file:
    screens_data = json.load(file)

matched_components = match_components(screens_data, components_data)
formatted_string = convert_matched_components_to_compressed_string(matched_components)

print(formatted_string)

Components:
[
    {
        "name": "Status bars",
        "type": ['Popup bar', 'Bottom bar', 'Top bar'],
        "size": ['Medium'],
        "count": ['1']
    },
    {
        "name": "Top navigation bar",
        "type": ['Text only', 'Centered', 'Left aligned'],
        "size": ['Default'],
        "count": ['1', '3', '0', '2']
    },
    {
        "name": "Text sections",
        "type": ['Body', 'Title'],
        "size": ['medium', 'large', 'small'],
        "count": ['Unknown']
    },
    {
        "name": "Accordian & collapsible",
        "type": ['Default'],
        "size": ['Medium'],
        "count": ['1']
    },
    {
        "name": "List",
        "type": ['Check box', 'Radio button', 'Icon only', 'Image landscape', 'Avatar', 'Button', 'Switch', 'Text'],
        "size": ['Medium'],
        "count": ['1', '3', '2']
    },
    {
        "name": "Seperator",
        "type": ['Title', 'Default'],
        "size": ['Medium'],
        "count": ['1']
    },
    {
        "name"

In [4]:
import json

def filter_screens_by_query(screens_data, query):
    """Filter screens based on the query."""
    filtered_screens = []
    for entry in screens_data:
        # Check if the query matches the entry's query
        if entry.get('query') == query:
            filtered_screens.extend(entry.get('screens', []))
    return filtered_screens

def extract_property_value(properties, keywords):
    """Extract property value from properties based on keywords."""
    for key, value in properties.items():
        if any(keyword in key.lower() for keyword in keywords):
            return value.get('value', 'Unknown')
    return 'Unknown'

def convert_json_to_formatted_string(screens_data, query):
    filtered_screens = filter_screens_by_query(screens_data, query)
    
    if not filtered_screens:
        print(f"No screens found for the query: {query}")
        return "Components = []\n"

    component_data = []

    for screen in filtered_screens:
        for comp in screen.get('components', []):
            comp_name = comp['name']
            properties = comp['properties']
            property_keywords = {
                'type': ['type'],
                'size': ['size'],
                'count': ['count'],
                'text_label': ['label', 'text label'],
                'icon': ['icon']
            }
            component_info = {prop: extract_property_value(properties, keys) for prop, keys in property_keywords.items()}
            component_info['name'] = comp_name
            component_data.append(component_info)

    if not component_data:
        print(f"No components found for the query: {query}")
        return "Components = []\n"

    formatted_string = "Components = [\n"
    for comp in component_data:
        formatted_string += "    {\n"
        formatted_string += f"        \"name\": \"{comp['name']}\",\n"
        formatted_string += f"        \"type\": \"{comp['type']}\",\n"
        formatted_string += f"        \"size\": \"{comp['size']}\",\n"
        formatted_string += f"        \"count\": \"{comp['count']}\",\n"
        formatted_string += f"        \"text_label\": \"{comp['text_label']}\",\n"
        formatted_string += f"        \"icon\": \"{comp['icon']}\"\n"
        formatted_string += "    },\n"
    formatted_string = formatted_string.rstrip(",\n") + "\n]\n"
    return formatted_string

screens_file_path = 'Prompt_Experiment/query-data.json'
with open(screens_file_path, 'r') as file:
    screens_data = json.load(file)

query = "Create a UI screen for to browse movies and TV shows and choose the best thing to watch"
formatted_string = convert_json_to_formatted_string(screens_data, query)
print(formatted_string)


Components = [
    {
        "name": "Status bars",
        "type": "Top bar",
        "size": "Medium",
        "count": "1",
        "text_label": "Unknown",
        "icon": "Unknown"
    },
    {
        "name": "Top navigation bar",
        "type": "Centered",
        "size": "Default",
        "count": "1",
        "text_label": "Unknown",
        "icon": "True"
    },
    {
        "name": "Tab menu",
        "type": "Badge",
        "size": "Default",
        "count": "Overflow",
        "text_label": "True",
        "icon": "False"
    },
    {
        "name": "Text sections",
        "type": "Title",
        "size": "small",
        "count": "Unknown",
        "text_label": "Unknown",
        "icon": "False"
    },
    {
        "name": "Carousel",
        "type": "Square",
        "size": "medium",
        "count": "Overflow",
        "text_label": "True",
        "icon": "False"
    },
    {
        "name": "Text sections",
        "type": "Title",
        "size": "small",
 

In [28]:
from dotenv import load_dotenv
import os
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

### System + User Prompt + JSON tool

In [27]:
from openai import OpenAI

client = OpenAI(api_key=f"{openai_api_key}")

tools = [
   {
    "type": "function",
    "function": {
        "name": "generate_ui_ux_json",
        "description": "Generate a JSON structure for UI/UX components and their properties",
        "parameters": {
            "type": "object",
            "properties": {
                "components": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "The name of the component"
                            },
                            "type": {
                                "type": "string",
                                "description": "The type of the component"
                            },
                            "size": {
                                "type": "string",
                                "description": "The size of the component"
                            },
                            "count": {
                                "type": "integer",
                                "description": "The count of the component"
                            },
                             "icon": {
                                "type": "string",
                                "description": "True or False. ICON to display, True or False"
                            },
                            "text label": {
                                "type": "string",
                                "description": "True or False. Whether to display the text label of the component or not"
                            },
                            
                        },
                        "required": ["name", "type", "size", "count"]
                    }
                }
            },
            "required": ["components"]
        }
                }
    }
]

system_prompt = """

<INST>

You are a highly skilled UI/UX designer, which can handle json structure very well and has 10 years of experience in ideating and developing great User friendly user interfaces. 
Here are the componets list available in JSON Strucutre, based on the components listed below and meaning of each thing, answer the user query


<JSON>

'' 
Components:
[
    {
        "name": "Status bars",
        "type": ['Popup bar', 'Bottom bar', 'Top bar'],
        "size": ['Medium'],
        "count": ['1']
    },
    {
        "name": "Top navigation bar",
        "type": ['Text only', 'Centered', 'Left aligned'],
        "size": ['Default'],
        "count": ['1', '3', '0', '2']
    },
    {
        "name": "Text sections",
        "type": ['Body', 'Title'],
        "size": ['medium', 'large', 'small'],
        "count": ['Unknown']
    },
    {
        "name": "Accordian & collapsible",
        "type": ['Default'],
        "size": ['Medium'],
        "count": ['1']
    },
    {
        "name": "List",
        "type": ['Check box', 'Radio button', 'Icon only', 'Image landscape', 'Avatar', 'Button', 'Switch', 'Text'],
        "size": ['Medium'],
        "count": ['1', '3', '2']
    },
    {
        "name": "Seperator",
        "type": ['Title', 'Default'],
        "size": ['Medium'],
        "count": ['1']
    },
    {
        "name": "Notification",
        "type": ['Toast', 'Snack bar'],
        "size": ['Medium'],
        "count": ['1']
    },
    {
        "name": "Banners/image",
        "type": ['Unknown'],
        "size": ['medium', 'large', 'small'],
        "count": ['1']
    },
    {
        "name": "Categories",
        "type": ['Badge', 'Tall', 'Rounded', 'Landscape'],
        "size": ['Large', 'Small', 'Medium'],
        "count": ['Overflow', '4', '5']
    },
    {
        "name": "Buttons",
        "type": ['Stepper', 'Button with text', 'Floating Button', 'Button', 'Social button'],
        "size": ['Large', 'Small'],
        "count": ['1', '3', '2']
    },
    {
        "name": "Input field",
        "type": ['Card input', 'Phone input', 'Text input'],
        "size": ['Large', 'Small'],
        "count": ['1', '2']
    },
    {
        "name": "Tab menu",
        "type": ['Badge', 'Line indicator'],
        "size": ['Default'],
        "count": ['Overflow', '2', '3']
    },
    {
        "name": "Search",
        "type": ['With button', 'Default'],
        "size": ['Large'],
        "count": ['1']
    },
    {
        "name": "Food Delivery cards",
        "type": ['Checkout card', 'Storefront', 'Grocery card', 'Product card', 'Coupon'],
        "size": ['Large', 'Small', 'Medium'],
        "count": ['2', 'overflow', '1', '3']
    },
    {
        "name": "Carousel",
        "type": ['Tall', 'Square', 'Landscape'],
        "size": ['medium', 'Large', 'Small'],
        "count": ['Overflow']
    },
    {
        "name": "Ecommerce cards",
        "type": ['Product card', 'Checkout card'],
        "size": ['Unknown'],
        "count": ['Overflow', '1', '2', '3']
    },
    {
        "name": "Bottom navigation bar",
        "type": ['Default'],
        "size": ['Medium'],
        "count": ['3', '4', '5']
    },
    {
        "name": "Pricing and paywall",
        "type": ['Table', 'Checkbox', 'paywall'],
        "size": ['Medium'],
        "count": ['2', '1', '3']
    },
    {
        "name": "Helper elements",
        "type": ['QR code', 'Overlay background', 'Over lay bar'],
        "size": ['Medium'],
        "count": ['1']
    },
    {
        "name": "Progress",
        "type": ['Slider', 'Steps', 'progress bar'],
        "size": ['Medium'],
        "count": ['2', '1', '3']
    },
    {
        "name": "Segmented menu",
        "type": ['Default'],
        "size": ['Medium'],
        "count": ['2', '3']
    },
    {
        "name": "map",
        "type": ['Default'],
        "size": ['Large', 'Small'],
        "count": ['1']
    },
    {
        "name": "Pagination",
        "type": ['circles'],
        "size": ['small'],
        "count": ['3']
    },
    {
        "name": "Video cards",
        "type": ['Scrollable video', 'thumbnail', 'Video player'],
        "size": ['Large', 'Small', 'Medium'],
        "count": ['1', 'overflow', '2']
    },
    {
        "name": "text and messaging",
        "type": ['Text-left', 'Text-right', 'chat input field'],
        "size": ['medium'],
        "count": ['1']
    },
    {
        "name": "Travel and booking",
        "type": ['Real estate card', 'Flight booking card'],
        "size": ['Small', 'Medium'],
        "count": ['Overflow', '1', '2']
    },
    {
        "name": "Social media cards",
        "type": ['Default', 'Blog card', 'Stories', 'Profile card', 'Thread'],
        "size": ['Medium'],
        "count": ['Overflow', '1']
    },
    {
        "name": "Calendar",
        "type": ['Event item', 'Calendar'],
        "size": ['Mediu', 'Small'],
        "count": ['1']
    },
    {
        "name": "Review card",
        "type": ['Default'],
        "size": ['Small', 'Medium'],
        "count": ['1']
    }
]

''

</JSON>

Your task is to generate a well-organized JSON structure for a UI/UX design based on the given components and properties. 
Follow best practices for UI/UX design to ensure a user-friendly and intuitive interface.

Stick to instructions and write a very well json structure with component names and properties.

Example JSON format:

Query : "Create a UI screen for to browse movies and TV shows and choose the best thing to watch"

Answer : 

<JSON>

Components = [
    {
        "name": "Status bars",
        "type": "Top bar",
        "size": "Medium",
        "count": "1",
    },
    {
        "name": "Top navigation bar",
        "type": "Centered",
        "size": "Default",
        "count": "1",
        "icon": "True"
    },
    {
        "name": "Tab menu",
        "type": "Badge",
        "size": "Default",
        "count": "Overflow",
        "text_label": "True",
        "icon": "False"
    },
    {
        "name": "Text sections",
        "type": "Title",
        "size": "small",
        "icon": "False"
    },
    {
        "name": "Carousel",
        "type": "Square",
        "size": "medium",
        "count": "Overflow",
        "text_label": "True",
        "icon": "False"
    },
    {
        "name": "Text sections",
        "type": "Title",
        "size": "small",
        "icon": "False"
    },
    {
        "name": "Carousel",
        "type": "Square",
        "size": "medium",
        "count": "Overflow",
        "text_label": "True",
        "icon": "False"
    },
    {
        "name": "Bottom navigation bar",
        "type": "Default",
        "size": "Medium",
        "count": "5",
        "text_label": "True",
    },
    {
        "name": "Status bars",
        "type": "Top bar",
        "size": "Medium",
        "count": "1",
        
    },
    {
        "name": "Top navigation bar",
        "type": "Left aligned",
        "size": "Default",
        "count": "2",
        "icon": "True"
    },
    {
        "name": "Tab menu",
        "type": "Badge",
        "size": "Default",
        "count": "Overflow",
        "text_label": "True",
        "icon": "False"
    },
    {
        "name": "Carousel",
        "type": "Landscape",
        "size": "medium",
        "count": "Overflow",
        "text_label": "True",
        "icon": "False"
    },
    {
        "name": "Text sections",
        "type": "Title",
        "size": "small",
        "icon": "False"
    },
    {
        "name": "List",
        "type": "Image landscape",
        "size": "Medium",
        "count": "1",
        "text_label": "False",
        "icon": "False"
    },
    {
        "name": "Seperator",
        "type": "Default",
        "size": "Medium",
        "count": "1",
    },
    {
        "name": "List",
        "type": "Image landscape",
        "size": "Medium",
        "count": "1",
        "text_label": "False",
        "icon": "False"
    },
    {
        "name": "Seperator",
        "type": "Default",
        "size": "Medium",
        "count": "1",
    },
    {
        "name": "List",
        "type": "Image landscape",
        "size": "Medium",
        "count": "1",
        "text_label": "False",
        "icon": "False"
    },
    {
        "name": "Seperator",
        "type": "Default",
        "size": "Medium",
        "count": "1",
    },
    {
        "name": "Text sections",
        "type": "Title",
        "size": "small",
        "icon": "False"
    },
    {
        "name": "Carousel",
        "type": "Square",
        "size": "medium",
        "count": "Overflow",
        "text_label": "True",
        "icon": "False"
    },
    {
        "name": "Bottom navigation bar",
        "type": "Default",
        "size": "Medium",
        "count": "5",
        "text_label": "True",
    },
    {
        "name": "Banners/image",
        "type": "Unknown",
        "size": "medium",
        "count": "1",
    },
    {
        "name": "Status bars",
        "type": "Top bar",
        "size": "Medium",
        "count": "1",
    },
    {
        "name": "Top navigation bar",
        "type": "Left aligned",
        "size": "Default",
        "count": "2",
        "icon": "True"
    },
    {
        "name": "Search",
        "type": "Default",
        "size": "Large",
        "count": "1",
        "icon": "True"
    },
    {
        "name": "Categories",
        "type": "Badge",
        "size": "Medium",
        "count": "Overflow",
        "text_label": "True",
        "icon": "False"
    },
    {
        "name": "Text sections",
        "type": "Title",
        "size": "small",
        "icon": "False"
    },
    {
        "name": "Ecommerce cards",
        "type": "Product card",
        "size": "Medium",
        "count": "2"
    },
    {
        "name": "Status bars",
        "type": "Top bar",
        "size": "Medium",
        "count": "1"
    },
    {
        "name": "Banners/image",
        "type": "Unknown",
        "size": "small",
        "count": "1"
    },
    {
        "name": "Text sections",
        "type": "Title",
        "size": "medium",
        "icon": "False"
    },
    {
        "name": "Tab menu",
        "type": "Badge",
        "size": "Default",
        "count": "2",
        "text_label": "True",
        "icon": "False"
    },
    {
        "name": "Input field",
        "type": "Text input",
        "size": "Large",
        "count": "1",
        "text_label": "False",
        "icon": "True"
    },
    {
        "name": "Text sections",
        "type": "Title",
        "size": "small",
        "icon": "False"
    },
    {
        "name": "List",
        "type": "Image landscape",
        "size": "Medium",
        "count": "1",
        "text_label": "False",
        "icon": "False"
    },
    {
        "name": "List",
        "type": "Image landscape",
        "size": "Medium",
        "count": "1",
        "text_label": "False",
        "icon": "False"
    },
    {
        "name": "Bottom navigation bar",
        "type": "Default",
        "size": "Medium",
        "count": "3",
        "text_label": "True"
    },
    {
        "name": "Buttons",
        "type": "Floating Button",
        "size": "Large",
        "count": "1",
        "text_label": "True"
    }
]

</JSON>

Stick to JSON Structure above and write properties well and acording to components mentioned above.

You have experience of UI design and know how components have to be made well structured, sometimes you have to mention about icon, text label whether they will be present or not via BOOLEAN values.
Acording to your expertise mention that as well for some components. Take reference from above JSON example.

Understand the meaning of these properties with the help of tools and write components and properties according to user needs.

</INST>

"""

user_query = " <INST> Design a UI for a shopping site </INST> "

### GPT4-o-mini testing

In [29]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.1,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ],
    tools=tools
)
print(completion)

ChatCompletion(id='chatcmpl-9rWzFNDHAex4YqUDmvmt355vOTFMI', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_R0vKaX15GRsclPU8XkJ02R5g', function=Function(arguments='{"components":[{"name":"Status bars","type":"Top bar","size":"Medium","count":1},{"name":"Top navigation bar","type":"Centered","size":"Default","count":1,"icon":"True"},{"name":"Search","type":"With button","size":"Large","count":1,"icon":"True"},{"name":"Categories","type":"Badge","size":"Medium","count":"Overflow","text label":"True","icon":"False"},{"name":"Carousel","type":"Square","size":"medium","count":"Overflow","text label":"True","icon":"False"},{"name":"Ecommerce cards","type":"Product card","size":"Medium","count":3},{"name":"Bottom navigation bar","type":"Default","size":"Medium","count":5,"text label":"True"},{"name":"Buttons","type":"Button with text","size":

In [30]:
import json

message_content = completion.choices[0].message.tool_calls[0].function.arguments
json_output = json.loads(message_content)
json_output

{'components': [{'name': 'Status bars',
   'type': 'Top bar',
   'size': 'Medium',
   'count': 1},
  {'name': 'Top navigation bar',
   'type': 'Centered',
   'size': 'Default',
   'count': 1,
   'icon': 'True'},
  {'name': 'Search',
   'type': 'With button',
   'size': 'Large',
   'count': 1,
   'icon': 'True'},
  {'name': 'Categories',
   'type': 'Badge',
   'size': 'Medium',
   'count': 'Overflow',
   'text label': 'True',
   'icon': 'False'},
  {'name': 'Carousel',
   'type': 'Square',
   'size': 'medium',
   'count': 'Overflow',
   'text label': 'True',
   'icon': 'False'},
  {'name': 'Ecommerce cards',
   'type': 'Product card',
   'size': 'Medium',
   'count': 3},
  {'name': 'Bottom navigation bar',
   'type': 'Default',
   'size': 'Medium',
   'count': 5,
   'text label': 'True'},
  {'name': 'Buttons',
   'type': 'Button with text',
   'size': 'Large',
   'count': 1,
   'text label': 'True'},
  {'name': 'Notification', 'type': 'Toast', 'size': 'Medium', 'count': 1},
  {'name': '

### GPT4-turbo testing

In [31]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "generate_ui_ux_json",
            "description": "Generate a JSON structure for UI/UX components and their properties",
            "parameters": {
                "type": "object",
                "properties": {
                    "components": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "name": {
                                    "type": "string",
                                    "description": "The name of the component"
                                },
                                "type": {
                                    "type": "string",
                                    "description": "The type of the component"
                                },
                                "size": {
                                    "type": "string",
                                    "description": "The size of the component"
                                },
                                "count": {
                                    "type": "string",
                                    "description": "The count of the component, which can be a number or a descriptive term like 'Overflow'"
                                },
                                "icon": {
                                    "type": "string",
                                    "description": "Indicates if an icon is displayed. 'True' or 'False'"
                                },
                                "text label": {
                                    "type": "string",
                                    "description": "Indicates if a text label is displayed. 'True' or 'False'"
                                }
                            },
                            "required": ["name", "type", "size", "count"]
                        }
                    }
                },
                "required": ["components"]
            }
        }
    }
]

In [32]:
completion0 = client.chat.completions.create(
    model="gpt-4-turbo",
    temperature=0.1,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ],
    tools=tools
)
print(completion0)

ChatCompletion(id='chatcmpl-9rX0bp9W1DnESsDc0nfBVLwOa8kys', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8H3Kmsy4J8H3l3w89EKiLq6f', function=Function(arguments='{"components":[{"name":"Top navigation bar","type":"Centered","size":"Default","count":"1","icon":"True"},{"name":"Search","type":"With button","size":"Large","count":"1","icon":"True"},{"name":"Categories","type":"Badge","size":"Medium","count":"Overflow","text label":"True","icon":"False"},{"name":"Ecommerce cards","type":"Product card","size":"Medium","count":"Overflow"},{"name":"List","type":"Image landscape","size":"Medium","count":"3","text label":"False","icon":"False"},{"name":"Pagination","type":"circles","size":"small","count":"3"},{"name":"Bottom navigation bar","type":"Default","size":"Medium","count":"5","text label":"True"}]}', name='generate_ui_ux_json'), type

In [33]:
message_content0 = completion0.choices[0].message.tool_calls[0].function.arguments
json_output0 = json.loads(message_content0)
json_output0

{'components': [{'name': 'Top navigation bar',
   'type': 'Centered',
   'size': 'Default',
   'count': '1',
   'icon': 'True'},
  {'name': 'Search',
   'type': 'With button',
   'size': 'Large',
   'count': '1',
   'icon': 'True'},
  {'name': 'Categories',
   'type': 'Badge',
   'size': 'Medium',
   'count': 'Overflow',
   'text label': 'True',
   'icon': 'False'},
  {'name': 'Ecommerce cards',
   'type': 'Product card',
   'size': 'Medium',
   'count': 'Overflow'},
  {'name': 'List',
   'type': 'Image landscape',
   'size': 'Medium',
   'count': '3',
   'text label': 'False',
   'icon': 'False'},
  {'name': 'Pagination', 'type': 'circles', 'size': 'small', 'count': '3'},
  {'name': 'Bottom navigation bar',
   'type': 'Default',
   'size': 'Medium',
   'count': '5',
   'text label': 'True'}]}

### GPT4-o testing

In [34]:
completion1 = client.chat.completions.create(
    model="gpt-4o",
    temperature=0.1,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ],
    tools=tools
)
print(completion1)

ChatCompletion(id='chatcmpl-9rX1wkHSeqD90M4fkQAg6OPlZZoVZ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_TDfH5y6tYMGeaCVSSgPhvrMx', function=Function(arguments='{"components":[{"name":"Status bars","type":"Top bar","size":"Medium","count":"1"},{"name":"Top navigation bar","type":"Left aligned","size":"Default","count":"1","icon":"True"},{"name":"Search","type":"Default","size":"Large","count":"1","icon":"True"},{"name":"Categories","type":"Badge","size":"Medium","count":"Overflow","text label":"True","icon":"False"},{"name":"Carousel","type":"Landscape","size":"Large","count":"Overflow","text label":"True","icon":"False"},{"name":"Text sections","type":"Title","size":"small","icon":"False"},{"name":"Ecommerce cards","type":"Product card","size":"Medium","count":"Overflow"},{"name":"Text sections","type":"Title","size":"small","icon"

In [35]:
message_content1 = completion1.choices[0].message.tool_calls[0].function.arguments
json_output1 = json.loads(message_content1)
json_output1

{'components': [{'name': 'Status bars',
   'type': 'Top bar',
   'size': 'Medium',
   'count': '1'},
  {'name': 'Top navigation bar',
   'type': 'Left aligned',
   'size': 'Default',
   'count': '1',
   'icon': 'True'},
  {'name': 'Search',
   'type': 'Default',
   'size': 'Large',
   'count': '1',
   'icon': 'True'},
  {'name': 'Categories',
   'type': 'Badge',
   'size': 'Medium',
   'count': 'Overflow',
   'text label': 'True',
   'icon': 'False'},
  {'name': 'Carousel',
   'type': 'Landscape',
   'size': 'Large',
   'count': 'Overflow',
   'text label': 'True',
   'icon': 'False'},
  {'name': 'Text sections', 'type': 'Title', 'size': 'small', 'icon': 'False'},
  {'name': 'Ecommerce cards',
   'type': 'Product card',
   'size': 'Medium',
   'count': 'Overflow'},
  {'name': 'Text sections', 'type': 'Title', 'size': 'small', 'icon': 'False'},
  {'name': 'Ecommerce cards',
   'type': 'Product card',
   'size': 'Medium',
   'count': 'Overflow'},
  {'name': 'Bottom navigation bar',
   't

### GPT4 testing

In [36]:
completion2 = client.chat.completions.create(
    model="gpt-4",
    temperature=0.1,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ],
    tools=tools
)
print(completion2)

ChatCompletion(id='chatcmpl-9rX2GZPQbykfxF94XaVzJAHGOliwk', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<JSON>\n\nComponents = [\n    {\n        "name": "Status bars",\n        "type": "Top bar",\n        "size": "Medium",\n        "count": "1",\n        "icon": "True",\n        "text_label": "True"\n    },\n    {\n        "name": "Top navigation bar",\n        "type": "Text only",\n        "size": "Default",\n        "count": "1",\n        "icon": "True",\n        "text_label": "True"\n    },\n    {\n        "name": "Search",\n        "type": "With button",\n        "size": "Large",\n        "count": "1",\n        "icon": "True",\n        "text_label": "True"\n    },\n    {\n        "name": "Categories",\n        "type": "Badge",\n        "size": "Medium",\n        "count": "Overflow",\n        "icon": "True",\n        "text_label": "True"\n    },\n    {\n        "name": "Ecommerce cards",\n        "type": "Product card",\n     

In [41]:
message_content2 = completion2.choices[0].message.content
print(message_content2)

<JSON>

Components = [
    {
        "name": "Status bars",
        "type": "Top bar",
        "size": "Medium",
        "count": "1",
        "icon": "True",
        "text_label": "True"
    },
    {
        "name": "Top navigation bar",
        "type": "Text only",
        "size": "Default",
        "count": "1",
        "icon": "True",
        "text_label": "True"
    },
    {
        "name": "Search",
        "type": "With button",
        "size": "Large",
        "count": "1",
        "icon": "True",
        "text_label": "True"
    },
    {
        "name": "Categories",
        "type": "Badge",
        "size": "Medium",
        "count": "Overflow",
        "icon": "True",
        "text_label": "True"
    },
    {
        "name": "Ecommerce cards",
        "type": "Product card",
        "size": "Medium",
        "count": "Overflow",
        "icon": "True",
        "text_label": "True"
    },
    {
        "name": "Pagination",
        "type": "circles",
        "size": "small",
 